In [1]:
# creating watermark or initial timestamp
import datetime
import database_handler
import lookups

In [2]:
db_session = database_handler.create_connection()

In [3]:

query = """
    CREATE TABLE IF NOT EXISTS public.etl_index
    (
        etl_last_run_date TIMESTAMP
    )
"""
database_handler.execute_query(query= query, db_session= db_session)


In [4]:

query = """
    SELECT 
        etl_last_run_date
    FROM public.etl_index
    ORDER BY 
        etl_last_run_date DESC
    LIMIT 1
"""
return_df = database_handler.return_data_as_df(file_executor= query, input_type= lookups.InputTypes.SQL, db_session= db_session)

if len(return_df) == 0:
    last_etl_run = datetime.datetime(1900,1,1)
else:
    last_etl_run = return_df['etl_last_run_date'].iloc[0]

print(last_etl_run)

2006-02-14 15:16:03


d:\Workspace\SE Factory\ETL_Project_2\database_handler.py:47: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return_dataframe = pd.read_sql_query(con= db_session, sql= file_executor)


In [5]:
insert_query = f"INSERT INTO public.etl_index (etl_last_run_date) VALUES ('{last_etl_run}')"
database_handler.execute_query(db_session, insert_query)

Extract

In [6]:
source_extract_query = f"SELECT * FROM public.rental WHERE last_update > '{last_etl_run}'"
df_rental = database_handler.return_data_as_df(file_executor= source_extract_query, input_type= lookups.InputTypes.SQL, db_session= db_session)


d:\Workspace\SE Factory\ETL_Project_2\database_handler.py:47: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return_dataframe = pd.read_sql_query(con= db_session, sql= file_executor)


In [7]:
# create_schema_query = """
#     CREATE SCHEMA IF NOT EXISTS reportingdw;
# """
# execute_query previous query
# create_table_query = database_handler.return_create_statement_from_df(df_rental,'reportingdw', 'stg_dvd_rental')
# execute_query previous query

Transformation

In [8]:
# we want to find out total number of rentals per customer since our last etl run
rental_per_customer = df_rental.groupby('customer_id').size().reset_index()
rental_per_customer.columns = ['customer_id','total_rentals']

In [9]:
rental_per_customer.head(1)

,customer_id,total_rentals


In [10]:
create_table_query = database_handler.return_create_statement_from_df(rental_per_customer, 'public','customer_rental_count')
database_handler.execute_query(db_session, create_table_query)

In [11]:
for index, row in rental_per_customer.iterrows():
    existing_count_query = f"SELECT total_rentals FROM public.customer_rental_count WHERE customer_id = {row['customer_id']};"
    existing_query_df = database_handler.return_data_as_df(file_executor= existing_count_query, input_type= lookups.InputTypes.SQL, db_session= db_session)
    length_df = len(existing_query_df)
    if length_df == 0:
        insert_query = f"INSERT INTO public.customer_rental_count (customer_id, total_rentals) VALUES ({row['customer_id']}, {row['total_rentals']})"
        database_handler.execute_query(db_session, insert_query)
    else:
        new_count = existing_query_df['total_rentals'][0] + row['total_rentals']
        update_query = f"UPDATE public.customer_rental_count SET total_rentals = {new_count} WHERE customer_id = {row['customer_id']};"
        database_handler.execute_query(db_session, update_query)


In [13]:
# update watermark
db_session = database_handler.create_connection()
last_rental_date = str(df_rental['rental_date'].max())
if len(df_rental) > 0:
    update_stmnt = f"UPDATE public.etl_index SET etl_last_run_date = '{last_rental_date}'"
    database_handler.execute_query(db_session, update_stmnt)

In [1]:
# pre-hook
# hook
# post-hook
from prehook import execute_prehook
execute_prehook()